In [7]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [8]:
from proj1_helpers import *
import os
data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'train.csv'
data_path = os.path.join(data_base_path, data_folder)
y, tX, ids = load_csv_data(data_path)

### Change later : Simple data description 
- all variables are floating point, except PRI_jet_num which is integer
- variables prefixed with PRI (for PRImitives) are “raw” quantities about the bunch collision as measured by the detector.
- variables prefixed with DER (for DERived) are quantities computed from the primitive features, which were selected by the physicists of ATLAS.
- it can happen that for some entries some variables are meaningless or cannot be computed; in this case, their value is −999.0, which is outside the normal range of all variables.

In [9]:
print('y shape = ' + str(y.shape) + '\ntX shape =' + str(tX.shape) + '\nids shape = ' + str(ids.shape))

y shape = (250000,)
tX shape =(250000, 30)
ids shape = (250000,)


### Missingness filter

In [18]:
temp = np.where(tX == -999, np.nan, tX)
missingness = np.sum(np.isnan(temp), axis = 0)/temp.shape[0]
print(missingness.shape)
print(missingness)

(30,)
[0.152456 0.       0.       0.       0.709828 0.709828 0.709828 0.
 0.       0.       0.       0.       0.709828 0.       0.       0.
 0.       0.       0.       0.       0.       0.       0.       0.399652
 0.399652 0.399652 0.709828 0.709828 0.709828 0.      ]


### Data pre-processing

We will first transform the categorical variable (PRI_jet_num) into four dummy variables

In [5]:
def transform_factor(x, column_idx, nlevels):
    new_var = np.zeros((x.shape[0], nlevels))
    
    for i in range(x.shape[0]):
        for j in range(nlevels):
            if x[i,column_idx] == j:
                new_var[j,0] = 1
                
    x = np.delete(x, column_idx, axis = 1)
    
    return np.concatenate((x, new_var), axis=1)
    
tX = transform_factor(x = tX, column_idx=22, nlevels =4)


## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)